# Sea Surface Temperature — NEMO GYRE

Analyse SST (`sosstsst`) from the 6-month GYRE simulation.
- Mean SST field
- SST temporal evolution
- Meridional SST gradient

In [ ]:
from pathlib import Path

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import nc_time_axis  # noqa: F401 — registers cftime support in matplotlib
import xarray as xr

plt.rcParams.update({"font.size": 16, "axes.titlesize": 18, "axes.labelsize": 14})

OUTPUT_DIR = Path("../output")

## Load simulation output

Load the recombined grid_T file and mesh_mask. Cell widths `e1t`/`e2t`
provide area weighting for spatial averages, and `tmask` masks land
and boundary cells.

In [ ]:
# Grid_T output — time axis decoded via CF-compliant 360_day calendar
ds = xr.open_mfdataset(sorted(OUTPUT_DIR.glob("*_grid_T.nc")))

# Mesh mask (time dim is degenerate, skip time decoding)
mask_ds = xr.open_dataset(OUTPUT_DIR / "mesh_mask.nc", decode_times=False)

# Surface ocean mask, excluding the outermost boundary ring
tmask_sfc = mask_ds.tmask.isel(time_counter=0, nav_lev=0)
ny, nx = tmask_sfc.sizes["y"], tmask_sfc.sizes["x"]
not_boundary = (
    (xr.DataArray(range(ny), dims="y") > 0)
    & (xr.DataArray(range(ny), dims="y") < ny - 1)
    & (xr.DataArray(range(nx), dims="x") > 0)
    & (xr.DataArray(range(nx), dims="x") < nx - 1)
)
interior = tmask_sfc.where(not_boundary, 0)

# Cell widths (m) for area-weighted averaging
e1t = mask_ds.e1t.isel(time_counter=0)
cell_area = e1t * mask_ds.e2t.isel(time_counter=0)

# Map projection centred on the GYRE domain
MARGIN = 0.5
proj = ccrs.Stereographic(central_longitude=-68, central_latitude=32)
extent = [
    float(ds.nav_lon.min()) - MARGIN, float(ds.nav_lon.max()) + MARGIN,
    float(ds.nav_lat.min()) - MARGIN, float(ds.nav_lat.max()) + MARGIN,
]

sst = ds["sosstsst"]
sst

## Time-mean SST

Average SST over the full simulation. Expect a north–south gradient
with warm subtropical gyre and cool subpolar gyre, plus western
boundary current signature.

In [ ]:
sst_mean = sst.mean("time_counter").where(interior)

fig, ax = plt.subplots(figsize=(8, 7), subplot_kw=dict(projection=proj))
ax.set_extent(extent, crs=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor="tan", edgecolor="k", linewidth=0.5)
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.gridlines(draw_labels=False, alpha=0.3)

pcm = ax.pcolormesh(
    ds.nav_lon.values, ds.nav_lat.values, sst_mean.values,
    shading="auto", cmap="RdYlBu_r", transform=ccrs.PlateCarree(),
)
fig.colorbar(pcm, ax=ax, label="SST (°C)", shrink=0.7)
ax.set_title("Mean SST")
fig.tight_layout()
fig.savefig("../figures/sst_mean.png", dpi=150, bbox_inches="tight")

## SST temporal evolution

Area-weighted domain-mean SST over time — tracks overall warming/cooling
trends during spin-up.

In [ ]:
weights = cell_area * interior
sst_domain_mean = sst.weighted(weights).mean(["y", "x"])

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(sst_domain_mean.time_counter, sst_domain_mean)
ax.set_xlabel("Time")
ax.set_ylabel("Domain-mean SST (°C)")
ax.set_title("NEMO GYRE — Domain-mean SST")
fig.tight_layout()

## Meridional SST gradient

Area-weighted zonal-mean SST as a function of latitude — shows the
subtropical-to-subpolar temperature contrast. Weighting by `e1t`
accounts for varying cell widths along the rotated GYRE grid.

In [ ]:
sst_time_mean = sst.mean("time_counter").where(interior)
sst_zonal = sst_time_mean.weighted(e1t).mean("x")

# Representative latitude for each y-row (mid-column value)
lat_1d = ds.nav_lat.isel(x=ds.sizes["x"] // 2)

fig, ax = plt.subplots(figsize=(6, 5))
ax.plot(sst_zonal, lat_1d)
ax.set_xlabel("SST (°C)")
ax.set_ylabel("Latitude")
ax.set_title("NEMO GYRE — Meridional SST Profile")
ax.grid(True, alpha=0.3)
fig.tight_layout()